# Terremotos en Mexico

Proceso ETL:

+ Extraccion de datos desde el servicio sismologico nacional de la UNAM: http://www.ssn.unam.mx
+ Transformacion necesaria de los datos
+ Carga de datos en base de datos y mapa.

In [1]:
import warnings
warnings.simplefilter('ignore')

import requests as req
import xmltodict
import re

import pandas as pd

**Extraccion**

In [2]:
url='http://www.ssn.unam.mx/rss/ultimos-sismos.xml'

data=req.get(url).content

In [12]:
xmltodict.parse(data)['rss']['channel']['item'][0]

OrderedDict([('title', '4.0, 89 km al SUROESTE de  PIJIJIAPAN, CHIS'),
             ('description',
              '<p>Fecha:2021-03-29 02:05:19 (Hora de M&eacute;xico)<br/>Lat/Lon: 15.1/-93.78<br/>Profundidad: 37.0 km </p>'),
             ('link',
              'http://www2.ssn.unam.mx:8080/jsp/localizacion-de-sismo.jsp?latitud=15.1&longitud=-93.78&prf=37.0 km&ma=4.0&fecha=2021-03-29&hora=02:05:19&loc=89 km al SUROESTE de  PIJIJIAPAN, CHIS  &evento=1'),
             ('geo:lat', '15.1'),
             ('geo:long', '-93.78')])

In [13]:
len(xmltodict.parse(data)['rss']['channel']['item'])

15

In [15]:
xmltodict.parse(data)['rss']['channel']['item'][0]['title']

'4.0, 89 km al SUROESTE de  PIJIJIAPAN, CHIS'

In [16]:
def get_data():   # funcion de extraccion de datos
    
    url='http://www.ssn.unam.mx/rss/ultimos-sismos.xml'  # viene de un RSS
    
    data=req.get(url).content
    
    return xmltodict.parse(data)

**Transformacion**

In [36]:
def get_dictio():
    
    d=get_data()
    
    res=[]   # rollo JSON
    
    lst=d['rss']['channel']['item']   # lista de elementos
    
    for e in lst:
        
        link=e['link']
        
        fecha=re.findall('fecha=\d+-\d+-\d+', link)[0].split('=')[1]
        hora=re.findall('hora=\d+:\d+:\d+', link)[0].split('=')[1]
        lat=re.findall('latitud=-?\d+.\d+', link)[0].split('=')[1]
        lng=re.findall('longitud=-?\d+.\d+', link)[0].split('=')[1]    
        inten=float(re.findall('ma=\d+.\d+', link)[0].split('=')[1])
        prof=re.findall('prf=\d+.\d+', link)[0].split('=')[1] 
        
        data={'Fecha': fecha, 
              'Hora': hora, 
              'Latitud': lat, 
              'Longitud': lng, 
              'Profundidad': prof, 
              'Intensidad': inten}
        
        res.append(data)
        
    return res

In [38]:
get_dictio()[0]

{'Fecha': '2021-03-29',
 'Hora': '02:05:19',
 'Latitud': '15.1',
 'Longitud': '-93.78',
 'Profundidad': '37.0',
 'Intensidad': 4.0}

In [39]:
lst=xmltodict.parse(data)['rss']['channel']['item'][0]['link'].split('?')[1].split('&')

for e in lst:
    print(e.split('='))

['latitud', '15.1']
['longitud', '-93.78']
['prf', '37.0 km']
['ma', '4.0']
['fecha', '2021-03-29']
['hora', '02:05:19']
['loc', '89 km al SUROESTE de  PIJIJIAPAN, CHIS  ']
['evento', '1']


In [42]:
{k:v for k,v in [e.split('=') for e in \
                xmltodict.parse(data)['rss']['channel']['item'][0]['link'].split('?')[1].split('&')]}

{'latitud': '15.1',
 'longitud': '-93.78',
 'prf': '37.0 km',
 'ma': '4.0',
 'fecha': '2021-03-29',
 'hora': '02:05:19',
 'loc': '89 km al SUROESTE de  PIJIJIAPAN, CHIS  ',
 'evento': '1'}

In [43]:
from urllib.parse import urlparse

u=urlparse(xmltodict.parse(data)['rss']['channel']['item'][0]['link'])
u.query

'latitud=15.1&longitud=-93.78&prf=37.0 km&ma=4.0&fecha=2021-03-29&hora=02:05:19&loc=89 km al SUROESTE de  PIJIJIAPAN, CHIS  &evento=1'

In [44]:
u.query.split('&')

['latitud=15.1',
 'longitud=-93.78',
 'prf=37.0 km',
 'ma=4.0',
 'fecha=2021-03-29',
 'hora=02:05:19',
 'loc=89 km al SUROESTE de  PIJIJIAPAN, CHIS  ',
 'evento=1']

In [45]:
{k:v for k,v in [e.split('=') for e in u.query.split('&')]}

{'latitud': '15.1',
 'longitud': '-93.78',
 'prf': '37.0 km',
 'ma': '4.0',
 'fecha': '2021-03-29',
 'hora': '02:05:19',
 'loc': '89 km al SUROESTE de  PIJIJIAPAN, CHIS  ',
 'evento': '1'}

**Carga base datos de postgres**

**Mapa (heatmap)**

**Heatmap animado**